In [1]:
using Random
using LinearAlgebra
using StateSpaceDynamics
using Distributions
using ProfileView 

n = 30000
Φ = randn(3, n)
Y = randn(1, n)
w = ones(n)

emission_1 = GaussianRegressionEmission(
    input_dim=3, output_dim=1, include_intercept=true,
    β=reshape([3.0, 2.0, 2.0, 3.0], :, 1), Σ=[1.0;;], λ=0.0
)
emission_2 = GaussianRegressionEmission(
    input_dim=3, output_dim=1, include_intercept=true,
    β=reshape([-4.0, -2.0, 3.0, 2.0], :, 1), Σ=[1.0;;], λ=0.0
)

A = [0.99 0.01; 0.05 0.95]
πₖ = [0.8; 0.2]
test_model = HiddenMarkovModel(K=2, A=A, πₖ=πₖ, B=[emission_1, emission_2])


for k in 1:test_model.K
    println("Allocations with transposes for iteration $k")
    @profview_allocs create_optimization(test_model.B[k], Φ', Y', w)
end


for k in 1:test_model.K
    println("Allocations without transposes for iteration $k")
    @profview_allocs create_optimization(test_model.B[k], Φ, Y, w)
end 

Allocations with transposes for iteration 1
Allocations with transposes for iteration 2
Allocations without transposes for iteration 1
Allocations without transposes for iteration 2
